In [2]:
import os

In [3]:
%pwd

'd:\\Courses\\Krish_Naik\\Chicken_Disease_Classification\\research'

In [4]:
# move one folder up
os.chdir("../")

In [1]:
%pwd

'd:\\Courses\\Krish_Naik\\Chicken_Disease_Classification\\research'

### 1. Update params.yaml with model parameters (DONE)

### 2. Update Entity

- Create a data class

In [7]:
from dataclasses import dataclass
from pathlib import Path

@dataclass(frozen=True)
class CallbacksConfig:
    root_dir: Path
    tensorboard_root_log_dir: Path
    model_checkpoint_dir: Path

### 3. Update Configuration Manager 

- Create root directory
- Create a sub directory
- initialize the data class object

In [6]:
from chicken_disease_classification.constants import *
from chicken_disease_classification.utils.common import read_yaml, create_directories

In [8]:
class ConfigurationManager:
    def __init__(
        self, 
        config_file_path: Path = CONFIG_FILE_PATH,
        params_file_path: Path = PARAMS_FILE_PATH
    ):
        # read base_model key in yaml file 
        self.config_file_path = read_yaml(config_file_path)
        # read params.yaml file
        self.params_file_path = read_yaml(params_file_path)
        
        # create directory -> "artifacts"; artifacts_root is present in config.yaml
        create_directories([self.config_file_path.artifacts_root])
    
    
    def get_callback_config(self) -> CallbacksConfig:
        
        callbacks_config = self.config_file_path.callbacks
        
        model_ckpt_dir = os.path.dirname(callbacks_config.model_checkpoint_dir)
        # create 2 sub directories in artifacts -> model_checkpoint_dir and tensorboard_root_log_dir
        create_directories([Path(model_ckpt_dir) , Path(callbacks_config.tensorboard_root_log_dir)])
        
        # create obj of BaseModelConfig class
        callbacks_config_obj = CallbacksConfig(
            root_dir = Path(callbacks_config.root_dir),
            tensorboard_root_log_dir = Path(callbacks_config.tensorboard_root_log_dir),
            model_checkpoint_dir = Path(callbacks_config.model_checkpoint_dir)
        )
        
        return callbacks_config_obj

### 4. Create Components

In [15]:
import os
import urllib.request as request
from zipfile import ZipFile
import tensorflow as tf
import time

In [13]:
class Callbacks:
    
    def __init__(self, config: CallbacksConfig):
        self.config = config
    
    
    @property
    def _create_tensorboard_callbacks(self):
        """create timestamp folder save all the logs in that folder
        """
        timestamp = time.strftime("%Y-%m-%d_%H-%M-%S")
        tensorboard_log_dir = os.path.join(self.config.tensorboard_root_log_dir, f"tensorborad_logs_at_{timestamp}")
        
        return tf.keras.callbacks.TensorBoard(log_dir = tensorboard_log_dir)
    
    
    @property
    def _create_ckpt_callbacks(self):
        return tf.keras.callbacks.ModelCheckpoint(
            filepath = self.config.model_checkpoint_dir,
            save_best_only = True
        )
    
    def get_tensorboard_ckpt_callbacks(self):
        return [self._create_tensorboard_callbacks, self._create_ckpt_callbacks]
        

### 5 Create Pipeline

In [16]:
try:
    config = ConfigurationManager()
    callbacks_config = config.get_callback_config()
    callbacks = Callbacks(config = callbacks_config)
    callback_list = callbacks.get_tensorboard_ckpt_callbacks()
    
except Exception as e:
    raise e

[2023-06-10 10:35:14,684: INFO: common: yaml file: config\config.yaml loaded successfully]
[2023-06-10 10:35:14,692: INFO: common: yaml file: params.yaml loaded successfully]
[2023-06-10 10:35:14,695: INFO: common: Created directory at: artifacts]
[2023-06-10 10:35:14,699: INFO: common: Created directory at: artifacts\callbacks\model_checkpoint_dir]
[2023-06-10 10:35:14,701: INFO: common: Created directory at: artifacts\callbacks\tensorboard_log_dir]


You get timsetamp folders like these:

In [17]:
timestamp = time.strftime("%Y-%m-%d_%H-%M-%S")
f"tensorborad_logs_at_{timestamp}"

'tensorborad_logs_at_2023-06-10_10-35-47'